In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
from scipy import stats
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, AgglomerativeClustering
from sklearn.metrics import pairwise_distances

In [2]:
ds = xr.open_dataset("./data/wind_europe_2015_2019.nc")

In [ ]:
selected_ds = ds # ds.sel(latitude=np.arange(40, 55.25, 0.25), longitude=np.arange(-5, 10.25, 0.25))
wind_speed = np.sqrt(selected_ds['u100']**2 + selected_ds['v100']**2).resample(time="1D").mean()

In [ ]:
cell_wind_speed = wind_speed.stack(cell=["latitude", "longitude"])
# mean_wind_speed = cell_wind_speed.groupby("cell").mean(...)
# std_wind_speed = cell_wind_speed.groupby("cell").std(...)
# norm_cell_wind_speed = (cell_wind_speed - mean_wind_speed) / std_wind_speed
norm_cell_wind_speed = cell_wind_speed.transpose("cell", "time")

In [ ]:
d = pairwise_distances(norm_cell_wind_speed, metric="correlation")

In [ ]:
clust = AgglomerativeClustering(n_clusters=None, distance_threshold=0.5, affinity='precomputed', linkage='complete').fit_predict(d) # n_clusters=6
res = xr.DataArray(clust, coords=[norm_cell_wind_speed['cell']], dims=['cell']).unstack("cell")

fig1 = plt.figure(figsize=(20,20))
ax = plt.axes(projection=ccrs.EqualEarth())
ax.coastlines(resolution='10m')
ax.margins(0)
ax.gridlines(draw_labels=True)
# ax.set_extent([-5, 10, 40, 55], crs=ccrs.PlateCarree())
plot1 = res.plot(cmap=plt.cm.tab20, transform=ccrs.PlateCarree(), cbar_kwargs={'shrink':0.4})

In [ ]:
a = norm_cell_wind_speed["cell"].values
dd = xr.DataArray(d, dims=["x", "y"], coords={"x": a, "y": a })
e = res.stack(cell=["latitude", "longitude"])

clust_inter = {}
for cluster in np.unique(clust):
    p = e[e == cluster]["cell"]
    p2 = dd.sel(x=p.values, y=p.values)
    clust_inter[cluster] = p2.quantile([0, 0.25, 0.5, 0.75]).values
clust_inter

In [ ]:
cell_wind_speed = wind_speed.stack(cell=["latitude", "longitude"])
mean_wind_speed = cell_wind_speed.groupby("cell").mean(...)
std_wind_speed = cell_wind_speed.groupby("cell").std(...)
m = (cell_wind_speed - mean_wind_speed) / std_wind_speed
m = m.transpose("cell", "time")
d2 = pairwise_distances(m)

In [ ]:
clust = AgglomerativeClustering(n_clusters=20, distance_threshold=None, affinity='precomputed', linkage='complete').fit_predict(d2) # n_clusters=6
res = xr.DataArray(clust, coords=[m['cell']], dims=['cell']).unstack("cell")

fig1 = plt.figure(figsize=(20,20))
ax = plt.axes(projection=ccrs.EqualEarth())
ax.coastlines(resolution='10m')
ax.margins(0)
ax.gridlines(draw_labels=True)
# ax.set_extent([-5, 10, 40, 55], crs=ccrs.PlateCarree())
plot1 = res.plot(cmap=plt.cm.tab20, transform=ccrs.PlateCarree(), cbar_kwargs={'shrink':0.4})

In [ ]:
clust = KMeans(n_clusters=8).fit_predict(m) # n_clusters=6
res = xr.DataArray(clust, coords=[m['cell']], dims=['cell']).unstack("cell")

fig1 = plt.figure(figsize=(20,20))
ax = plt.axes(projection=ccrs.EqualEarth())
ax.coastlines(resolution='10m')
ax.margins(0)
ax.gridlines(draw_labels=True)
# ax.set_extent([-5, 10, 40, 55], crs=ccrs.PlateCarree())
plot1 = res.plot(cmap=plt.cm.tab20, transform=ccrs.PlateCarree(), cbar_kwargs={'shrink':0.4})